In [ ]:
import numpy as np
import pickle
import glob
from sklearn.mixture import  GaussianMixture
from sklearn import mixture,metrics
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('seaborn-whitegrid')
import seaborn as sns
from scipy.stats import multivariate_normal
from itertools import chain
import os
import pandas as pd
import threading

In [ ]:
file_path = '/content/drive/MyDrive/Research Project/random-word-vectors-main'
file_repos = ['TM_Project/04']#,'words_pkl_750_1','words_pkl_750_2','words_pkl_750_3','words_pkl_12_10']
#file_repos = ['words_pkl_12_10']

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path = [file_path+'/'+irepos+'/' for irepos in file_repos]
path

['/content/drive/MyDrive/Research Project/random-word-vectors-main/TM_Project/04/']

In [ ]:
files = []
for ifiles in path:
  print(ifiles)
  files.append(glob.glob(ifiles+"*.pkl"))
  
files = list(chain(*files))

/content/drive/MyDrive/Research Project/random-word-vectors-main/TM_Project/04/


In [ ]:
sts = {}
for f in files:
  sts[f.split('/')[-1].split('.')[0]]=0

words = []
word_Stats = {}
threadList = []


for ifile in files:
  words.append(ifile.split('/')[-1].split('.')[0])

words = sorted(set(words))
print(words)

['advisory', 'assistance', 'belief', 'communication', 'court', 'data', 'date', 'department', 'documents', 'education', 'finance', 'good', 'government', 'head', 'health', 'island', 'light', 'medical', 'office', 'school', 'space', 'support', 'target', 'territory', 'women', 'workshops']


In [ ]:
def func_runMOG(word,arr,layer):
  mog_path = ''
  with open (file_path+'/combined_vectors/MOG_Clusters_901_1100_25.txt','a')as txtfile:
    k_Components = arr.shape[0]  
    k = min(11,k_Components)
    K = range(1,k)
    mogs = [GaussianMixture(n_components=k,max_iter=500).fit(arr) for k in K]
    bics = [mog.bic(arr) for mog in mogs]
    aics = [mog.aic(arr) for mog in mogs]
    llks = [mog.score(arr) for mog in mogs]
    ss = []
    for mog in mogs:
      y = mog.predict(arr)
      try:
        ss.append(round(metrics.silhouette_score(arr,y),4))
      except:
        #print(mog.n_components)
        pass
    print(word,":",layer,":",k_Components," :",np.argmax(ss)+2," :",ss[np.argmax(ss)],':', ss)
    result = '{} ,{}, {}, {}, {}, {}\n'.format(word,layer,k_Components,np.argmax(ss)+2,ss[np.argmax(ss)],ss)
  #return result
    txtfile.write(result)
  txtfile.close()  

In [ ]:
def multi_threading(a,b):
  for keys in list(word_Stats.keys())[a:b]:
    t = threading.Thread(target = func_runMOG,args=(word,word_Stats[keys],keys,))
    threadList.append(t)
    t.start()
  for tr in threadList:
    tr.join()

In [ ]:
import os
import warnings

# Ignore FutureWarning from numpy
warnings.simplefilter(action='ignore', category=FutureWarning)

import keras.backend as K
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Allow growth of GPU memory, otherwise it will always look like all the memory is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True) 

In [ ]:
cnt = 0
with tf.device('/device:GPU:0'):
  for word in words:
    for repo in file_repos:
      path = file_path+'/'+repo+'/'+word+'.pkl'
      #print(path)
      index = 0
      if not (os.path.exists(path)):
        continue
      if word !='xxx':
        with (open(path, "rb")) as openfile:
            while True:
                try:
                  word_vectors = pickle.load(openfile)
                  vec = list(word_vectors.keys())[2:]
                  for ivec in vec:
                    if sts[word]==0 and index == 0 :
                      arr = word_vectors[ivec].reshape(25,1024)
                      index+=1
                    else:
                      arr = np.row_stack((arr,word_vectors[ivec].reshape(25,1024)))
                      index+=1 
                except EOFError:
                  break
        sts[word] =arr.shape[0]/25
    if sts[word]>5:
      #word_Stats[word] = arr
      cnt = cnt +1
      print(cnt, word, sts[word])
      arr_layer=np.zeros([1,1024])
      for layers in range(25):
        l_count =0
        layer = layers
        arr_layer=np.zeros([1,1024])    
        while True:
          arr_layer = np.row_stack((arr_layer,arr[layer,]))
          #print('positions:',layer)        
          layer+=25
          l_count+=1
          if l_count==sts[word]:
            #print(layer_index)
            arr_layer = arr_layer[1:,]
            word_Stats[layers] = arr_layer
            #func_runMOG(word,arr_layer)
            break
      #multi_threading(0,5)
      #multi_threading(5,10)
      #multi_threading(10,15)
      #multi_threading(15,20)
      #multi_threading(20,25)


1 advisory 1191.0
2 assistance 4605.0
3 belief 68.0
4 communication 1213.0
5 court 343.0
6 data 416.0
7 date 125.0
8 department 2057.0
9 documents 145.0
10 education 4792.0
11 finance 697.0
12 good 1732.0
13 government 1701.0
14 head 107.0
15 health 4892.0
16 island 101.0
17 light 174.0
18 medical 493.0
19 office 1122.0
20 school 712.0
21 space 2522.0
22 support 7977.0


In [ ]:
with open (path+'/MOG_Clusters_100_25.txt','w')as txtfile:
  for word, arr in word_Stats.items():
    k_Components = arr.shape[0]  
    k = min(11,k_Components)
    K = range(1,k)
    mogs = [GaussianMixture(n_components=k,max_iter=500).fit(arr) for k in K]
    bics = [mog.bic(arr) for mog in mogs]
    aics = [mog.aic(arr) for mog in mogs]
    llks = [mog.score(arr) for mog in mogs]
    ss = []
    for mog in mogs:
      y = mog.predict(arr)
      try:
        ss.append(round(metrics.silhouette_score(arr,y),4))
      except:
        #print(mog.n_components)
        pass
    print(word,":",k_Components," :",np.argmax(ss)+2," :",ss[np.argmax(ss)],':', ss)
    result = '{} , {}, {}, {}, {}\n'.format(word,k_Components,np.argmax(ss)+2,ss[np.argmax(ss)],ss)
    txtfile.write(result)

respect : 337  : 4  : 0.1709 : [0.0939, 0.1519, 0.1709, 0.1176, 0.1494, 0.146, 0.0992, 0.0929, 0.0938, 0.1111, 0.0998, 0.073, 0.0869, 0.0884]
respond : 36  : 2  : 0.1225 : [0.1225, 0.0756, 0.054, 0.0246, 0.0452, 0.0244, 0.0264, 0.0448, 0.0528, 0.0546, 0.0618, 0.0336, 0.0611, 0.0332]
response : 174  : 2  : 0.1581 : [0.1581, 0.1379, 0.1193, 0.0957, 0.0887, 0.0846, 0.078, 0.0851, 0.0451, 0.0882, 0.0729, 0.0712, 0.094, 0.071]
responsibility : 144  : 3  : 0.0671 : [0.0501, 0.0671, 0.062, 0.0558, 0.0605, 0.0528, 0.0652, 0.0555, 0.0569, 0.0545, 0.0545, 0.0404, 0.0458, 0.0464]
responsible : 12  : 2  : 0.2084 : [0.2084, 0.0591, 0.0511, 0.1008, 0.1275, 0.0671, 0.1152, 0.0988, 0.0774, 0.0528]
rest : 44  : 2  : 0.2044 : [0.2044, 0.089, 0.1576, 0.0472, 0.1355, 0.051, 0.0593, 0.0695, 0.0667, 0.0476, 0.0589, 0.0821, 0.0435, 0.0414]
retain : 8  : 5  : 0.0436 : [0.0336, 0.0384, 0.032, 0.0436, 0.0336, 0.0313]
return : 138  : 2  : 0.1219 : [0.1219, 0.1043, 0.1036, 0.0951, 0.0943, 0.0875, 0.0559, 0.0663, 